In [70]:

from datetime import datetime
import pandas as pd
import numpy as np
import io
import os

In [71]:
def ler_csv(filepath, sep=",") -> pd.DataFrame: 
    return pd.read_csv(filepath, sep=sep)

In [72]:
orders = ler_csv("/Users/ismael/Documents/Projects-Data_Engineering/Nosql/ecomerce-pipeline-project/data/olist/olist_orders_dataset.csv")
order_items = ler_csv("/Users/ismael/Documents/Projects-Data_Engineering/Nosql/ecomerce-pipeline-project/data/olist/olist_order_items_dataset.csv")
customers = ler_csv("/Users/ismael/Documents/Projects-Data_Engineering/Nosql/ecomerce-pipeline-project/data/olist/olist_customers_dataset.csv")
payments = ler_csv("/Users/ismael/Documents/Projects-Data_Engineering/Nosql/ecomerce-pipeline-project/data/olist/olist_order_payments_dataset.csv")
reviews = ler_csv("/Users/ismael/Documents/Projects-Data_Engineering/Nosql/ecomerce-pipeline-project/data/olist/olist_order_reviews_dataset.csv")

#list of dataframes
dfs = [orders, order_items, customers, payments, reviews]
df_names = ["orders", "order_items", "customers", "payments", "reviews"]


In [73]:
for name, df in zip(df_names, dfs):
    print(f"DataFrame shape {name}: {df.shape}")
    print("-" * 50)

DataFrame shape orders: (99441, 8)
--------------------------------------------------
DataFrame shape order_items: (112650, 7)
--------------------------------------------------
DataFrame shape customers: (99441, 5)
--------------------------------------------------
DataFrame shape payments: (103886, 5)
--------------------------------------------------
DataFrame shape reviews: (99224, 7)
--------------------------------------------------


In [74]:
#verifycaing the duplicates
for name, df in zip(df_names, dfs):
    print(f"DataFrame {name} has {df.duplicated().sum()} duplicates")
    print("-" * 50)

DataFrame orders has 0 duplicates
--------------------------------------------------
DataFrame order_items has 0 duplicates
--------------------------------------------------
DataFrame customers has 0 duplicates
--------------------------------------------------
DataFrame payments has 0 duplicates
--------------------------------------------------
DataFrame reviews has 0 duplicates
--------------------------------------------------


In [75]:
# printing the columns of each dataframe
for name, df in zip(df_names, dfs):
    print(f"DataFrame {name} columns: {df.columns.tolist()}")
    print("-" * 100)

DataFrame orders columns: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
----------------------------------------------------------------------------------------------------
DataFrame order_items columns: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']
----------------------------------------------------------------------------------------------------
DataFrame customers columns: ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
----------------------------------------------------------------------------------------------------
DataFrame payments columns: ['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']
---------------------------------------------------------------------------------------------

In [76]:
#verricying the null values
for name, df in zip(df_names, dfs):
    print(f"DataFrame {name} has {df.isnull().sum().sum()} null values")
    print("-" * 50)

DataFrame orders has 4908 null values
--------------------------------------------------
DataFrame order_items has 0 null values
--------------------------------------------------
DataFrame customers has 0 null values
--------------------------------------------------
DataFrame payments has 0 null values
--------------------------------------------------
DataFrame reviews has 145903 null values
--------------------------------------------------


In [77]:
# transforming null values to for example 0 or np.nan
for name, df in zip(df_names, dfs):
    df.fillna('sem informação', inplace=True)  # or use np.nan if you prefer
    print(f"DataFrame {name} null values after filling: {df.isnull().sum().sum()}")
    print("-" * 50)

DataFrame orders null values after filling: 0
--------------------------------------------------
DataFrame order_items null values after filling: 0
--------------------------------------------------
DataFrame customers null values after filling: 0
--------------------------------------------------
DataFrame payments null values after filling: 0
--------------------------------------------------
DataFrame reviews null values after filling: 0
--------------------------------------------------


In [78]:
# verifying the values unique in each column
for name, df in zip(df_names, dfs):
    print(f"DataFrame {name} unique values in each column:")
    for col in df.columns:
        print(f"{col}: {df[col].nunique()}")
    print("-" * 50)

DataFrame orders unique values in each column:
order_id: 99441
customer_id: 99441
order_status: 8
order_purchase_timestamp: 98875
order_approved_at: 90734
order_delivered_carrier_date: 81019
order_delivered_customer_date: 95665
order_estimated_delivery_date: 459
--------------------------------------------------
DataFrame order_items unique values in each column:
order_id: 98666
order_item_id: 21
product_id: 32951
seller_id: 3095
shipping_limit_date: 93318
price: 5968
freight_value: 6999
--------------------------------------------------
DataFrame customers unique values in each column:
customer_id: 99441
customer_unique_id: 96096
customer_zip_code_prefix: 14994
customer_city: 4119
customer_state: 27
--------------------------------------------------
DataFrame payments unique values in each column:
order_id: 99440
payment_sequential: 29
payment_type: 5
payment_installments: 24
payment_value: 29077
--------------------------------------------------
DataFrame reviews unique values in eac

In [79]:
#verifing the data types of each column
for name, df in zip(df_names, dfs):
    print(f"DataFrame {name} data types:")
    print(df.dtypes)
    print("-" * 50)

DataFrame orders data types:
order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object
--------------------------------------------------
DataFrame order_items data types:
order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object
--------------------------------------------------
DataFrame customers data types:
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object
-------------------------------------------------

In [80]:
from collections import defaultdict

# Etapa 1: Merge das tabelas principais
df = pd.merge(orders, order_items, on="order_id", how="left")
df = pd.merge(df, customers, on="customer_id", how="left")
df = pd.merge(df, payments, on="order_id", how="left")
df = pd.merge(df, reviews, on="order_id", how="left")

print("Merge finalizado. Shape:", df.shape)


Merge finalizado. Shape: (119143, 28)


In [82]:
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,...,1.0,credit_card,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,sem informação,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,...,3.0,voucher,1.0,2.00,a54f0611adc9ed256b57ede6b6eb5114,4.0,sem informação,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,...,2.0,voucher,1.0,18.59,a54f0611adc9ed256b57ede6b6eb5114,4.0,sem informação,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,...,1.0,boleto,1.0,141.46,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,...,1.0,credit_card,3.0,179.12,e73b67b67587f7644d5bd1a52deb1b01,5.0,sem informação,sem informação,2018-08-18 00:00:00,2018-08-22 19:07:58


In [81]:
print("Orders columns:", orders.columns.tolist())
print("Order Items columns:", order_items.columns.tolist())


Orders columns: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
Order Items columns: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']


In [83]:
def group_orders(df):
    from collections import defaultdict
    orders_dict = defaultdict(lambda: {
        "order_id": None,
        "customer": {},
        "items": [],
        "payment": {},
        "review": {},
        "status": None,
        "order_date": None
    })

    for _, row in df.iterrows():
        order = orders_dict[row['order_id']]
        order["order_id"] = row["order_id"]
        order["status"] = row["order_status"]
        order["order_date"] = row["order_purchase_timestamp"]

        order["customer"] = {
            "customer_id": row["customer_id"],
            "city": row["customer_city"],
            "state": row["customer_state"]
        }

        order["items"].append({
            "product_id": row["product_id"],
            "seller_id": row["seller_id"],
            "price": float(row["price"]) if not pd.isnull(row["price"]) else 0.0,
            "freight_value": float(row["freight_value"]) if not pd.isnull(row["freight_value"]) else 0.0
        })

        order["payment"] = {
            "type": row["payment_type"],
            "installments": int(row["payment_installments"]) if not pd.isnull(row["payment_installments"]) else 0,
            "value": float(row["payment_value"]) if not pd.isnull(row["payment_value"]) else 0.0
        }

        order["review"] = {
            "score": int(row["review_score"]) if not pd.isnull(row["review_score"]) else None,
            "comment_title": row["review_comment_title"] if pd.notnull(row["review_comment_title"]) else "",
            "comment_message": row["review_comment_message"] if pd.notnull(row["review_comment_message"]) else ""
        }

    return list(orders_dict.values())


In [84]:
orders_json = group_orders(df)

print(f"Total de pedidos agrupados: {len(orders_json)}")

# Ver 1 pedido formatado
import json
print(json.dumps(orders_json[0], indent=2, ensure_ascii=False))


Total de pedidos agrupados: 99441
{
  "order_id": "e481f51cbdc54678b7cc49136f2d6af7",
  "customer": {
    "customer_id": "9ef432eb6251297304e76186b10a928d",
    "city": "sao paulo",
    "state": "SP"
  },
  "items": [
    {
      "product_id": "87285b34884572647811a353c7ac498a",
      "seller_id": "3504c0cb71d7fa48d967e0e4c94d59d9",
      "price": 29.99,
      "freight_value": 8.72
    },
    {
      "product_id": "87285b34884572647811a353c7ac498a",
      "seller_id": "3504c0cb71d7fa48d967e0e4c94d59d9",
      "price": 29.99,
      "freight_value": 8.72
    },
    {
      "product_id": "87285b34884572647811a353c7ac498a",
      "seller_id": "3504c0cb71d7fa48d967e0e4c94d59d9",
      "price": 29.99,
      "freight_value": 8.72
    }
  ],
  "payment": {
    "type": "voucher",
    "installments": 1,
    "value": 18.59
  },
  "review": {
    "score": 4,
    "comment_title": "sem informação",
    "comment_message": "Não testei o produto ainda, mas ele veio correto e em boas condições. Apenas a